# FEW SHOT - CREATE (agora com memória)

CREATE significa:
- Caracterização: Definição do papel ou persona do modelo.
- Requisição: Solicitação principal.
- Exemplos: Amostras para guiar a resposta.
- Ajustes: Modificações ou restrições.
- Tipo: Formato ou estilo da saída.
- Extras: Informações adicionais relevantes.

In [1]:
# Imports necessários
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.graph import START, MessagesState, StateGraph

from IPython.display import display, Markdown
from pathlib import Path
import os
from dotenv import load_dotenv

In [2]:
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [4]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

## Aqui vou usar o template CREATE

In [5]:
prompt = """
**C — Caracterização**
Você é um engenheiro de QA experiente, especialista em testes funcionais escritos no estilo Gherkin.

**R — Requisição**
Crie um plano de testes funcional baseado na história técnica descrita ao final do prompt.

**E — Exemplos**
A seguir estão exemplos genéricos que demonstram **exatamente o formato** em que os testes devem ser escritos:

```
**Teste 1**: Verificar acesso ao painel  
**Cenário**: Usuário autenticado acessa painel principal  
**Dado** que o usuário está logado  
**Quando** acessa a URL "/painel"  
**Então** ele deve visualizar a página com o título "Painel Principal"

**Teste 2**: Campo obrigatório não preenchido  
**Cenário**: Validação de campo obrigatório  

**Dado** que o usuário deixa o campo "nome" em branco  
**Quando** tenta salvar o formulário  
**Então** o sistema deve exibir a mensagem "Campo obrigatório"
```

> (Os exemplos não correspondem ao requisito da tela de login, mas **demonstram claramente o padrão desejado de resposta**.)

**A — Ajustes**
Mantenha esse mesmo padrão de escrita nos seus testes: título, cenário, e blocos Gherkin. Evite mudar o estilo ou a sequência.

**T — Tipo**
Formato: texto estruturado conforme os exemplos, com blocos Given–When–Then (Dado / Quando / Então) bem definidos.

**E — Extras**
Considere casos positivos (fluxos válidos) e negativos (erros, limites, exceções). O objetivo é garantir cobertura total dos requisitos.

Requisitos:
* O campo de nome de usuário deve aceitar apenas e-mails válidos.
* A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.
* O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.
* O link "lembrar senha" deve levar o usuário para a tela correspondente.
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


**C — Caracterização**
Você é um engenheiro de QA experiente, especialista em testes funcionais escritos no estilo Gherkin.

**R — Requisição**
Crie um plano de testes funcional baseado na história técnica descrita ao final do prompt.

**E — Exemplos**
A seguir estão exemplos genéricos que demonstram **exatamente o formato** em que os testes devem ser escritos:

```
**Teste 1**: Verificar acesso ao painel  
**Cenário**: Usuário autenticado acessa painel principal  
**Dado** que o usuário está logado  
**Quando** acessa a URL "/painel"  
**Então** ele deve visualizar a página com o título "Painel Principal"

**Teste 2**: Campo obrigatório não preenchido  
**Cenário**: Validação de campo obrigatório  

**Dado** que o usuário deixa o campo "nome" em branco  
**Quando** tenta salvar o formulário  
**Então** o sistema deve exibir a mensagem "Campo obrigatório"
```

> (Os exemplos não correspondem ao requisito da tela de login, mas **demonstram claramente o padrão desejado de resposta**.)

**A — Ajustes**
Mantenha esse mesmo padrão de escrita nos seus testes: título, cenário, e blocos Gherkin. Evite mudar o estilo ou a sequência.

**T — Tipo**
Formato: texto estruturado conforme os exemplos, com blocos Given–When–Then (Dado / Quando / Então) bem definidos.

**E — Extras**
Considere casos positivos (fluxos válidos) e negativos (erros, limites, exceções). O objetivo é garantir cobertura total dos requisitos.

Requisitos:
* O campo de nome de usuário deve aceitar apenas e-mails válidos.
* A senha deve ter entre 8 e 20 caracteres, contendo letras maiúsculas, minúsculas, números e caracteres especiais.
* O sistema deve permitir no máximo 3 tentativas de login. Após isso, um captcha deve ser exibido.
* O link "lembrar senha" deve levar o usuário para a tela correspondente.


----

*Resposta:*

----

**Teste 1**: Validação de e-mail no campo de nome de usuário  
**Cenário**: Usuário preenche um e-mail válido  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "nome de usuário" com "usuario@exemplo.com"  
**Então** o sistema deve aceitar o e-mail como válido  

**Teste 2**: Validação de e-mail no campo de nome de usuário  
**Cenário**: Usuário preenche um e-mail inválido  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "nome de usuário" com "usuario-exemplo.com"  
**Então** o sistema deve exibir a mensagem "Por favor, insira um e-mail válido"  

**Teste 3**: Validação da senha  
**Cenário**: Usuário preenche a senha correta  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "senha" com "Senha@123"  
**Então** o sistema deve aceitar a senha como válida  

**Teste 4**: Validação da senha  
**Cenário**: Usuário preenche a senha com menos de 8 caracteres  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "senha" com "abc"  
**Então** o sistema deve exibir a mensagem "A senha deve ter entre 8 e 20 caracteres"  

**Teste 5**: Validação da senha  
**Cenário**: Usuário preenche a senha com mais de 20 caracteres  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "senha" com "SenhaMuitoLonga123@"  
**Então** o sistema deve exibir a mensagem "A senha deve ter entre 8 e 20 caracteres"  

**Teste 6**: Validação da senha  
**Cenário**: Usuário preenche a senha sem caracteres especiais  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "senha" com "Senha123"  
**Então** o sistema deve exibir a mensagem "A senha deve conter letras maiúsculas, minúsculas, números e caracteres especiais"  

**Teste 7**: Tentativas de login  
**Cenário**: Usuário tenta logar com credenciais incorretas três vezes  
**Dado** que o usuário está na tela de login  
**Quando** tenta fazer login com "usuario@exemplo.com" e "senhaErrada" três vezes seguidas  
**Então** o sistema deve exibir um captcha  

**Teste 8**: Link de lembrar senha  
**Cenário**: Usuário clica no link "lembrar senha"  
**Dado** que o usuário está na tela de login  
**Quando** clica no link "lembrar senha"  
**Então** o usuário deve ser redirecionado para a tela de recuperação de senha  

----

## Vamos testar a memória

In [6]:
prompt = """
Adicione mais casos de testes, por favor.
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


Adicione mais casos de testes, por favor.


----

*Resposta:*

----

Claro! Aqui estão mais casos de teste para garantir uma cobertura ainda melhor dos requisitos:

**Teste 9**: Login com credenciais corretas  
**Cenário**: Usuário loga com e-mail e senha válidos  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "nome de usuário" com "usuario@exemplo.com"  
**E** preenche o campo "senha" com "Senha@123"  
**E** clica no botão "Entrar"  
**Então** o sistema deve redirecioná-lo para a página inicial do aplicativo  

**Teste 10**: Tentativas de login com e-mail inválido  
**Cenário**: Usuário tenta logar com um e-mail inválido  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "nome de usuário" com "usuario-exemplo.com"  
**E** preenche o campo "senha" com "Senha@123"  
**E** clica no botão "Entrar"  
**Então** o sistema deve exibir a mensagem "Por favor, insira um e-mail válido"  

**Teste 11**: Login após captcha exibido  
**Cenário**: Usuário tenta logar após 3 tentativas incorretas  
**Dado** que o usuário já completou 3 tentativas de login sem sucesso  
**E** o captcha está sendo exibido  
**Quando** o usuário preenche o captcha corretamente e tenta logar novamente com credenciais corretas  
**Então** o sistema deve redirecioná-lo para a página inicial do aplicativo  

**Teste 12**: Login após captcha exibido - tentativa incorreta  
**Cenário**: Usuário completa o captcha mas tenta logar com credenciais incorretas  
**Dado** que o usuário já completou 3 tentativas de login sem sucesso  
**E** o captcha está sendo exibido  
**Quando** o usuário preenche o captcha corretamente e tenta logar novamente com "usuario@exemplo.com" e "senhaErrada"  
**Então** o sistema deve exibir a mensagem "Credenciais incorrectas"  

**Teste 13**: Campo de nome de usuário vazio  
**Cenário**: Usuário tenta fazer login sem preencher o campo de nome de usuário  
**Dado** que o usuário está na tela de login  
**Quando** deixa o campo "nome de usuário" em branco  
**E** preenche o campo "senha" com "Senha@123"  
**E** clica no botão "Entrar"  
**Então** o sistema deve exibir a mensagem "Campo obrigatório"  

**Teste 14**: Campo de senha vazio  
**Cenário**: Usuário tenta fazer login sem preencher o campo de senha  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "nome de usuário" com "usuario@exemplo.com"  
**E** deixa o campo "senha" em branco  
**E** clica no botão "Entrar"  
**Então** o sistema deve exibir a mensagem "Campo obrigatório"  

**Teste 15**: Acessar página de recuperação de senha e retorno  
**Cenário**: Usuário volta da tela de recuperação de senha  
**Dado** que o usuário está na tela de recuperação de senha  
**Quando** clica no link "Voltar ao login"  
**Então** o sistema deve redirecioná-lo para a tela de login  

**Teste 16**: Validação de senha sem número  
**Cenário**: Usuário preenche a senha sem números  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "senha" com "Senha@Abc"  
**Então** o sistema deve exibir a mensagem "A senha deve conter letras maiúsculas, minúsculas, números e caracteres especiais"  

**Teste 17**: Validação de senha sem letra maiúscula  
**Cenário**: Usuário preenche a senha sem letras maiúsculas  
**Dado** que o usuário está na tela de login  
**Quando** preenche o campo "senha" com "senha123@"  
**Então** o sistema deve exibir a mensagem "A senha deve conter letras maiúsculas, minúsculas, números e caracteres especiais"  

Esses novos testes visam fortalecer a cobertura e aumentar a robustez das validações presentes no sistema de login.

----

## Pedindo para alterar o formato

In [7]:
prompt = """
Agora coloque no formato tabela, conforme exemplo abaixo:

| Nº do Teste | Título                     | Pré-condições        | Ação                                             | Resultado Esperado                         |
| ----------- | -------------------------- | -------------------- | ------------------------------------------------ | ------------------------------------------ |
| 1           | E-mail inválido não aceito | Tela de login aberta | Inserir "usuariosemarroba.com" e clicar em Login | Mensagem de erro "E-mail inválido" exibida |
| 2           | Senha curta bloqueada      | Tela de login aberta | Inserir "abc123" como senha e clicar em Login    | Mensagem de erro "Senha inválida" exibida  |
"""

display(Markdown(f"*Prompt:*"))
display(Markdown(f"----"))
display(Markdown(f"{prompt}"))
display(Markdown(f"----"))

input_messages = [HumanMessage(prompt)]
output = app.invoke({"messages": input_messages}, config)
resposta = output["messages"][-1].content

display(Markdown(f"*Resposta:*"))
display(Markdown(f"----"))
display(Markdown(f"{resposta}"))
display(Markdown(f"----"))

*Prompt:*

----


Agora coloque no formato tabela, conforme exemplo abaixo:

| Nº do Teste | Título                     | Pré-condições        | Ação                                             | Resultado Esperado                         |
| ----------- | -------------------------- | -------------------- | ------------------------------------------------ | ------------------------------------------ |
| 1           | E-mail inválido não aceito | Tela de login aberta | Inserir "usuariosemarroba.com" e clicar em Login | Mensagem de erro "E-mail inválido" exibida |
| 2           | Senha curta bloqueada      | Tela de login aberta | Inserir "abc123" como senha e clicar em Login    | Mensagem de erro "Senha inválida" exibida  |


----

*Resposta:*

----

Claro! Aqui está o conjunto de testes em formato de tabela:

| Nº do Teste | Título                                    | Pré-condições              | Ação                                                   | Resultado Esperado                                                       |
| ----------- | ---------------------------------------- | -------------------------- | ------------------------------------------------------ | ------------------------------------------------------------------------ |
| 1           | E-mail válido aceito                    | Tela de login aberta       | Inserir "usuario@exemplo.com" e clicar em Login       | Login realizado com sucesso e redirecionamento para a página inicial     |
| 2           | E-mail inválido não aceito              | Tela de login aberta       | Inserir "usuario-exemplo.com" e clicar em Login       | Mensagem de erro "Por favor, insira um e-mail válido" exibida            |
| 3           | Senha válida aceita                      | Tela de login aberta       | Inserir "Senha@123" e clicar em Login                  | Login realizado com sucesso e redirecionamento para a página inicial     |
| 4           | Senha curta bloqueada                   | Tela de login aberta       | Inserir "abc" como senha e clicar em Login             | Mensagem de erro "A senha deve ter entre 8 e 20 caracteres" exibida      |
| 5           | Senha longa bloqueada                   | Tela de login aberta       | Inserir "SenhaMuitoLonga123@" como senha              | Mensagem de erro "A senha deve ter entre 8 e 20 caracteres" exibida      |
| 6           | Senha sem caractere especial            | Tela de login aberta       | Inserir "Senha123" como senha                          | Mensagem de erro "A senha deve conter letras maiúsculas, minúsculas, números e caracteres especiais" exibida |
| 7           | Excesso de tentativas de login          | Tela de login aberta       | Tentar logar 3 vezes com "usuario@exemplo.com" e "senhaErrada" | Captcha exibido                                                          |
| 8           | Navegação para lembrar senha             | Tela de login aberta       | Clicar no link "lembrar senha"                         | Redirecionamento para a tela de recuperação de senha                    |
| 9           | Login com credenciais corretas          | Tela de login aberta       | Inserir "usuario@exemplo.com" e "Senha@123" e clicar em Login | Login realizado com sucesso e redirecionamento para a página inicial     |
| 10          | Tentativa de login com e-mail inválido  | Tela de login aberta       | Inserir "usuario-exemplo.com" e "Senha@123" e clicar em Login | Mensagem de erro "Por favor, insira um e-mail válido" exibida            |
| 11          | Login após captcha exibido               | Após 3 tentativas de login sem sucesso | Inserir captcha corretamente e tentar logar com credenciais corretas | Login realizado com sucesso e redirecionamento para a página inicial     |
| 12          | Login após captcha - tentativa incorreta | Após 3 tentativas de login sem sucesso | Inserir captcha corretamente e tentar logar com "usuario@exemplo.com" e "senhaErrada" | Mensagem de erro "Credenciais incorretas" exibida                        |
| 13          | Campo de nome de usuário vazio          | Tela de login aberta       | Deixar campo "nome de usuário" em branco e preencher "Senha@123" e clicar em Login | Mensagem de erro "Campo obrigatório" exibida                             |
| 14          | Campo de senha vazio                    | Tela de login aberta       | Preencher "usuario@exemplo.com" em "nome de usuário" e deixar "senha" em branco e clicar em Login | Mensagem de erro "Campo obrigatório" exibida                             |
| 15          | Acesso de volta da recuperação de senha | Tela de recuperação de senha| Clicar no link "Voltar ao login"                        | Redirecionamento para a tela de login                                     |
| 16          | Senha sem número bloqueada              | Tela de login aberta       | Inserir "Senha@Abc" e clicar em Login                  | Mensagem de erro "A senha deve conter letras maiúsculas, minúsculas, números e caracteres especiais" exibida |
| 17          | Senha sem letra maiúscula bloqueada     | Tela de login aberta       | Inserir "senha123@" e clicar em Login                   | Mensagem de erro "A senha deve conter letras maiúsculas, minúsculas, números e caracteres especiais" exibida |

Essa tabela abrange os diversos cenários de teste, incluindo casos positivos e negativos para garantir a funcionalidade do sistema de login.

----